## ジョブシーケンス問題
計算時間が知られている$N$個のジョブをM個のコンピュータに処理させるときに、ジョブを各コンピュータに配分して、各コンピュータの実行時間を最小にする問題です。 

TAYTAN SDKのインストール

In [ ]:
!pip install -q git+https://github.com/tytansdk/tytan.git

In [ ]:
!pip install --quiet networkx matplotlib

### 問題設定について
コンピュータはN個あります。また、コンピュータの番号を指定する文字を$a$とします。 
ジョブはM個あり、ジョブを指定する文字を$i$とします。 $i$番目のジョブというような言い方を使います。  
各ジョブの実行時間は$L_i$で表します。$i$はジョブの番号です。この$L_i$はあらかじめ知られているとします。  
最適化したい変数は、$a$番目のコンピュータで$i$番目のジョブを行うときに1となり、  
そうでないときに0となる変数$x_{a,i}$とします。

問題は、各コンピューターの実行時間を最小化したいのですが、QUBO式で表現するために工夫が入ります。  
問題を実行時間が最大のコンピュータの実行時間を最小化する問題として考え、 さらに、   
1番目のコンピュータの実行時間が最大の時間になるとして、1番目のコンピュータの実行時間を最小化する問題に変換します. 


### QUBO式
コスト関数について、説明します。  
コスト関数は、大きく分けて4つの構成となります。　　

1つ目は、1番目のコンピュータの実行時間です。  
2つ目は、コンピュータ1が最大の実行時間となるコンピュータである条件です。  
3つ目は、ジョブ$i$はただ一つのコンピュータでのみ実行される条件です。  
4つ目は、2つ目の制約条件を満たすために追加する変数に関する制約条件です。  



1つ目の、1番目のコンピュータの実行時間を定式化すると以下になります。
$$
H_A = \sum_{i} L_i x_{1,i}
$$

2つ目は、コンピュータ1が最大の実行時間となるコンピュータである条件は、
$$
H_B =\sum_{a=2} \{ \sum_n^M n y_{n,a} - \sum_{i}( L_i x_{1,i} - L_i x_{a,i})\}
$$　　
　　
  
ここで、補助変数$y_{n,a}$は、コンピュータ1とコンピュータ$a$の実行時間の差が$n$であるときに1、そうでない場合は0となる補助変数です。  
定数$M$はノード1との実行時間の差の最大値で, 任意で決めるハイパーパラメータです。  


3つ目は、ジョブ$i$はただ一つのコンピュータでのみ実行される条件 
$$
H_C = \sum_{i} (1 - \sum_{a} x_{a,i})^2 
$$

4つ目は、変数$y_{n,a}$がaに対してただ1つだけ1になる条件です。  
また、変数の一つだけが1となる変数をone-hot変数といいます。  
$$
H_D = \sum_{a} (1 - \sum_{n} y_{n,a})^2 
$$


## 問題設定
$10$個のジョブがある. 
各ジョブの実行時間は, 

|ジョブ |  1   |  2   |  3   |  4   |  5   |  6   |  7   |  8   |  9   | 10   |
| ----  | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| 時間  |  1   |  2   |  3   |  4   |  5   |  6   |  7   |  8   |  9   | 10   |

上記のジョブを3つのノードに分けて実行するときに, 最大実行時間のノードを最小化する. 

| COM|  1   |  2   |  3   |  4   | 合計 |
| ----  | ---- | ---- | ---- | ---- | ---- |
|COM1|  4   |  7   |  8   |  0   |  19  |
|COM2|  1   |  2   |  5   |  10  |  18  |
|COM3|  3   |  6   |  9   |  0   |  18  |

In [1]:
import numpy as np
import sympy as sym
from tytan import symbols, Compile, sampler

#40量子ビット
J=10 #ジョブの数
N=3  #コンピュータの数
M=5  #最大実行時間の差

#量子ビットJ*N=30
#one hot用の量子ビット (N-1)*M=10
q = symbols("q_{0:40}") # qubo

#ジョブの実行時間
L=np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])


#式A 1番目のCOMの実行時間
HA=0
for i in range(J):#ジョブ数
    HA += L[i]*q[i]

#式B
HB=0
for a in range(1,N): #コンピュータ数
    tmp=0
    for n in range(M):#最大実行時間
        tmp += (n+1)*q[J*N+(a-1)*M+n]
          
    for i in range(J):#ジョブ数
        tmp += -L[i]*(q[i]-q[a*J+i])
    HB += tmp**2
#式C
HC=0
for i in range(J):#ジョブ数
    tmp=0
    for a in range(N):#コンピュータ数
        tmp += q[a*J+i]
    HC += (1-tmp)**2


#式D onehot 
HD=0
for i in range(N-1):#コンピュータ数
    tmp=0
    for a in range(M):
        tmp += q[J*N+i*M+a]
    HD += (1-tmp)**2

#式E
HE=0
tmp=0
for i in range(J):
    for a in range(N):
        tmp += L[i]*q[a*J+i]
HE = (np.sum(L) - tmp)**2

#式をつなげる
H = 10*HA+20*HB+7*HC+3*HD+3*HE

# Compileクラスを使用して、QUBOを取得
Q, offset = Compile(H).get_qubo()

#print(Q)

# サンプラーを選択
solver = sampler.SASampler()

#クラウドサンプラーの場合
#API_KEY = "KjGh2i75pUNXvihIMgNiQfMAQmbU8iVe01tjStnYJWMb8AS1SSys3SPpMN_q_8gl"
#solver = sampler.NQSSampler()
#result = solver.run(Q, api_key=API_KEY)

# 計算
result = solver.run(Q, shots=1000)

#print("Sample =", result[0][0])
print("Cost =", result[0][1] + offset)

#量子ビットの取り出し
qbit=np.zeros(40)
i=0
for key, value in result[0][0].items():
    key=key.replace('q_{', '')
    key=key.replace('}', '')
    qbit[int(key)]=value
    i=i+1
print(qbit)
print('コンピュータ1',qbit[0:10], np.sum(L*qbit[0:10]))
print('コンピュータ2',qbit[10:20], np.sum(L*qbit[10:20]))
print('コンピュータ3',qbit[20:30], np.sum(L*qbit[20:30]))

print('onehot1',qbit[30:35])
print('onehot2',qbit[35:40])
#制約条件
B=0
for a in range(1,N): #コンピュータ数
    tmp=0
    for n in range(M):#最大実行時間
        tmp += (n+1)*qbit[J*N+(a-1)*M+n]
        
    for i in range(J):#ジョブ数
        tmp += -L[i]*(qbit[i]-qbit[a*J+i])
    B += tmp
print('制約B',B)

Cost = 190.000000000000
[1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1.
 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
コンピュータ1 [1. 0. 0. 0. 0. 0. 0. 1. 0. 1.] 19.0
コンピュータ2 [0. 0. 0. 0. 1. 1. 1. 0. 0. 0.] 18.0
コンピュータ3 [0. 1. 1. 1. 0. 0. 0. 0. 1. 0.] 18.0
onehot1 [1. 0. 0. 0. 0.]
onehot2 [1. 0. 0. 0. 0.]
制約B 0.0


In [ ]:
シュミレーションで制約Bを満たす結果が得られませんでした。
D-WAVEのシュミレータで確認してみます。

In [370]:
# D-wave シュミレーション例
import numpy as np
import sympy as sym
#from tytan import *
from pyqubo import Array, Constraint
import neal
q = Array.create('q', shape=40, vartype='BINARY') 

#今回は4量子ビットのみ
J=10 #ジョブの数
N=3  #コンピュータの数
M=5  #最大実行時間の差

#量子ビットJ*N
#one hot用の量子ビット (N-1)*M=6
#q = sym.symbols("q_{0:36}") # qubo

#ジョブの実行時間
L=np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])


#式A 1番目のCOMの実行時間
HA=0
for i in range(J):#ジョブ数
    HA += -L[i]*q[i]

#式B
HB=0
for a in range(1,N): #コンピュータ数
    tmp=0
    for n in range(M):#最大実行時間
        tmp += (n+1)*q[J*N+(a-1)*M+n]
          
    for i in range(J):#ジョブ数
        tmp += -L[i]*(q[i]-q[a*J+i])
    HB += tmp**2
    
#式C
HC=0
for i in range(J):#ジョブ数
    tmp=0
    for a in range(N):#コンピュータ数
        tmp += q[a*J+i]
    HC += (1-tmp)**2


#式D onehot 
HD=0
for i in range(N-1):#コンピュータ数
    tmp=0
    for a in range(M):
        tmp += q[J*N+i*M+a]
    HD += (1-tmp)**2

#式をつなげる
H = 2*HA+2*Constraint(HB,'HB')+7*Constraint(HC,'HC')+3*Constraint(HD,'HD')

#D-Waveの形式に変換
model = H.compile()
bqm = model.to_bqm()

#計算実行
sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm, num_reads=10)
#print(sampleset)

decoded_samples = model.decode_sampleset(sampleset)
best_sample = min(decoded_samples, key=lambda x: x.energy)
#print("Sample =", best_sample.sample) 
print("Cost =", best_sample.energy)
print("Break =", best_sample.constraints(only_broken=True))

#量子ビットの取り出し
qbit=np.zeros(40)
i=0
for key, value in best_sample.sample.items():
    key=key.replace('q[', '')
    key=key.replace(']', '')
    qbit[int(key)]=value
    i=i+1
    
print(qbit)
print('コンピュータ1',qbit[0:10])
print('コンピュータ2',qbit[10:20])
print('コンピュータ3',qbit[20:30])

print('onehot1',qbit[30:35])
print('onehot2',qbit[35:40])

#制約条件
B=0
for a in range(1,N): #コンピュータ数
    tmp=0
    for n in range(M):#最大実行時間
        tmp += (n+1)*qbit[J*N+(a-1)*M+n]
        
    for i in range(J):#ジョブ数
        tmp += -L[i]*(qbit[i]-qbit[a*J+i])
    B += tmp
print('制約B',B)

Cost = -42.0
Break = {}
[1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1.
 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0.]
コンピュータ1 [1. 0. 1. 0. 0. 0. 1. 0. 0. 1.]
コンピュータ2 [0. 1. 0. 0. 0. 1. 0. 0. 1. 0.]
コンピュータ3 [0. 0. 0. 1. 1. 0. 0. 1. 0. 0.]
onehot1 [0. 0. 0. 1. 0.]
onehot2 [0. 0. 0. 1. 0.]
制約B 0.0


コンピューター1 = {1,3,7,10} 実行時間21  
コンピューター2 = {2,6,9} 　 実行時間17  
コンピューター3 = {4,5,8}    実行時間17  

## ナップザック問題
価値が分かっている$N$個の荷物があります. この荷物をナップザックの最大容量$W$を超えない範囲で入れるときに, 価値の合計を最大化する組み合わせを探す問題です. 

| 荷物 |  1   |  2   |  3   |  4   |  5   |
| ---- | ---- | ---- | ---- | ---- | ---- |
| 重さ |  4   |  7   |  1   |  3   |  5   |
| 価値 |  10  |  13  |  7   |  9   |  8   |

ナップザックの最大容量が10のとき, 
荷物1,3,4を入れると容量が8で価値26となります. 
総当たりで探索すると$2^N$回の探索が必要になります. (NP困難)

### QUBO式

$i$番目の荷物をナップザックに入れる場合は, $x_i=1$, 入れない場合は, $x_i=0$とします. 

$i$番目の荷物の容量を$w_i$, 荷物の価値を$c_i$とします. 

ナップザックに入れる荷物の価値を最大化したいので, QUBOコスト関数は, 
$$
H_A = -\sum^N_{i=1} c_i x_i
$$
です. 

QUBO式にマイナスがついているのは, QUBO関数は最小に向かうために, 関数を最大化したいときにはマイナスを付けます. 

制約式は, ナップザックに入れる荷物が総体積以下になる条件
$$
\sum^N_{i=1} w_i x_i < W
$$
ですが, QUBO式は不等式を扱えません. 

そのために補助バイナリ変数$y_n$を導入します. 
$y_n$はナップザックに入れる荷物の容量の合計が, $n$のときに$y_n=1$となる変数です. 
ナップザックの最大容量は$W$なので$n$は0から$N$までの値を取ります. 

$y_n$は荷物の容量の合計値なので, $N$個ある$y_n$はただ1つのみ1になる制約があります. 
制約式を書くと, 

$$
H_B = (1-\sum^W_{n=1} y_n)^2
$$

$y_n=1$になるときに, ナップザックの荷物の容量の合計$ \sum^N_{i=1} w_i x_i = n$になるような条件式を追加します. 
$$
H_C = (\sum^W_{n=1}n y_n -  \sum^N_{i=1} w_i x_i)^2
$$
変数$y_{n,a}$がaに対してただ1つだけ1になる条件です。  
また、変数の一つだけが1となる変数をone-hot変数といいます。  


## 問題設定

ナップザックの最大容量$W=12$のとき, 以下の$N=5$個の荷物を考えます. 

| 荷物 |  1   |  2   |  3   |  4   |  5   |
| ---- | ---- | ---- | ---- | ---- | ---- |
| 重さ |  3   |  4   |  6   |  1   |  5   |
| 価値 |  6   |  7   |  8   |  1   |  4   |

必要なビット変数は, $N+W=5+12=17$

In [34]:
import numpy as np
from tytan import *
import sympy as sym

#17量子ビットのみ
N=5
W=12

M=N+W
q = sym.symbols("q_{0:17}") 

#重さ
w=np.array([3, 4, 6, 1, 5])

#価値
c=np.array([6 ,7, 8, 1, 4])

#式A
HA=0
for i in range(N):
    HA += -c[i]*q[i]

#式B
HB=0
for i in range(N,N+W):
    HB +=q[i]
HB = HB**2
    
#式C
HC=0
for i in range(N,N+W):
    HC += (i-4)*q[i]
for j in range(N):
    HC += -w[j]*q[j]

HC=(HC)**2


#式をつなげる
H = 1*HA+15*HB+7*HC

# Compileクラスを使用して、QUBOを取得
Q, offset = qubo.Compile(H).get_qubo()

#print(Q)

# サンプラーを選択
solver = sampler.SASampler()

#クラウドサンプラーの場合
#API_KEY = "KjGh2i75pUNXvihIMgNiQfMAQmbU8iVe01tjStnYJWMb8AS1SSys3SPpMN_q_8gl"
#solver = sampler.NQSSampler()
#result = solver.run(Q, api_key=API_KEY)

# 計算
result = solver.run(Q, shots=1000)

#print("Sample =", result[0][0])
print("Cost =", result[0][1] + offset)

#量子ビットの取り出し
qbit=np.zeros(40)
i=0
for key, value in result[0][0].items():
    key=key.replace('q_{', '')
    key=key.replace('}', '')
    qbit[int(key)]=value
    i=i+1
print('q=',qbit[0:N])
print('重さ',w[0]*qbit[0]+w[1]*qbit[1]+w[2]*qbit[2]+w[3]*qbit[3]+w[4]*qbit[4])
print('コスト',c[0]*qbit[0]+c[1]*qbit[1]+c[2]*qbit[2]+c[3]*qbit[3]+c[4]*qbit[4])
print('onehot=',qbit[N:N+W])

Cost = -2.0
q= [1. 1. 0. 0. 1.]
重さ 12.0
コスト 17.0
onehot= [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


In [ ]:
 荷物1,2,5を入れるときに制約を満たす結果が得られました。その時の価値は17となっています。

In [31]:
# D-WAVEシュミレータ
import numpy as np
from tytan import *
import sympy as sym
from pyqubo import Array, Constraint
import neal

#17量子ビット
N=5
W=12

M=N+W
q = Array.create('q', shape=17, vartype='BINARY') 
#重さ
w=np.array([3, 4, 6, 1, 5])

#価値
c=np.array([6 ,7, 8, 1, 4])

#式A
HA=0
for i in range(N):
    HA += -c[i]*q[i]

#式B
HB=0
for i in range(N,N+W):
    HB +=q[i]
HB = (HB-1)**2
    
#式C
HC=0
for i in range(N,N+W):
    HC += (i-4)*q[i]
for j in range(N):
    HC += -w[j]*q[j]

HC=(HC)**2


#式をつなげる
H = 1*HA+15*Constraint(HB,'HB')+7*Constraint(HC,'HC')

#D-Waveの形式に変換
model = H.compile()
bqm = model.to_bqm()

#計算実行
sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm, num_reads=10)
#print(sampleset)

decoded_samples = model.decode_sampleset(sampleset)
best_sample = min(decoded_samples, key=lambda x: x.energy)
#print("Sample =", best_sample.sample) 
print("Cost =", best_sample.energy)
print("Break =", best_sample.constraints(only_broken=True))

#量子ビットの取り出し
qbit=np.zeros(40)
i=0
for key, value in best_sample.sample.items():
    key=key.replace('q[', '')
    key=key.replace(']', '')
    qbit[int(key)]=value
    i=i+1
print('q=',qbit[0:N])
print('重さ',w[0]*qbit[0]+w[1]*qbit[1]+w[2]*qbit[2]+w[3]*qbit[3]+w[4]*qbit[4])
print('コスト',c[0]*qbit[0]+c[1]*qbit[1]+c[2]*qbit[2]+c[3]*qbit[3]+c[4]*qbit[4])
print('onehot=',qbit[N:N+W])

Cost = -15.0
Break = {}
q= [1. 0. 1. 1. 0.]
重さ 10.0
コスト 15.0
onehot= [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


 荷物1,3,4を入れるときに制約を満たす結果が得られました。そのときの価値は15となりました。